In [2]:
import keras
from keras import Sequential
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import PIL
import os, shutil
keras.__version__

Using TensorFlow backend.


'2.2.0'

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8521045532001646679
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7930327860
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3353997639404478814
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:0a:00.0, compute capability: 6.1"
]


In [4]:
!pwd

/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50


In [5]:
original_dataset_dir = '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data'

# The directory where we will
# store our smaller dataset
base_dir = '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small'

categories = ['cats', 'dogs']
str_train_val_test = ['train','validation', 'test']
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    print('Created directory: ', base_dir)
        
for dir_type in str_train_val_test:
    train_test_val_dir = os.path.join(base_dir, dir_type)

    if not os.path.exists(train_test_val_dir):
        os.mkdir(train_test_val_dir)

    for category in categories:
        dir_type_category = os.path.join(train_test_val_dir, category)
        
        if not os.path.exists(dir_type_category):
            os.mkdir(dir_type_category)
            print('Created directory: ', dir_type_category)
  

In [6]:
directories_dict = {} # To store directory paths for data subsets.

for range_, dir_type in zip([range(1000), range(1000, 1500), range(1500, 2000)], str_train_val_test):
    for _, category in enumerate(categories):
        fnames = [category[:-1] + "." + str(i) + "." + "jpg" for i in range_]
        

        if _ == 0:
                directories_dict[str(dir_type)+ "_dir"] = os.path.join(base_dir, 
                                                                       dir_type, 
                                                                       )                          
        for _, fname in enumerate(fnames):
            
            if _ == 0:
                directories_dict[str(dir_type)+ "_" +str(category) + "_dir"] =  os.path.join(base_dir, 
                                                                                         dir_type, 
                                                                                         category)   
#             source = os.path.join(original_dataset_dir, fname)
#             destination = os.path.join(base_dir, dir_type, category, fname)
            
            
#             shutil.copyfile(source, destination)

In [7]:
directories_dict

{'test_cats_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/test/cats',
 'test_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/test',
 'test_dogs_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/test/dogs',
 'train_cats_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train/cats',
 'train_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train',
 'train_dogs_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train/dogs',
 'validation_cats_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation/cats',
 'validation_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation',
 'validation_dogs_dir': '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation/dogs'}

In [8]:
print('Total training cat images:', len(os.listdir(directories_dict['train_cats_dir'])))
print('Total training dog images:', len(os.listdir(directories_dict['train_cats_dir'])))
print("-"*32)
print('Total test cat images:', len(os.listdir(directories_dict['test_cats_dir'])))
print('Total test dog images:', len(os.listdir(directories_dict['test_dogs_dir'])))
print("-"*32)
print('Total validation cat images:', len(os.listdir(directories_dict['validation_cats_dir'])))
print('Total validation dog images:', len(os.listdir(directories_dict['validation_dogs_dir'])))


Total training cat images: 1000
Total training dog images: 1000
--------------------------------
Total test cat images: 500
Total test dog images: 500
--------------------------------
Total validation cat images: 500
Total validation dog images: 500


In [9]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
 
train_generator =  test_datagen.flow_from_directory(       # This is the target directory
        directories_dict['train_dir'],
        # All images will be resized to 150x150
        target_size=(197, 197),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        directories_dict['validation_dir'],
        target_size=(197, 197),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
from keras.applications import ResNet50

conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(197, 197, 3))

In [11]:
# from keras.applications import VGG16

# conv_base = VGG16(weights='imagenet',
#                   include_top=False,
#                   input_shape=(150, 150, 3))

In [12]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 203, 203, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [13]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [14]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3' 

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/10
100/100 [==============================] - 39s 390ms/step - loss: 0.2438 - acc: 0.8990 - val_loss: 0.1328 - val_acc: 0.9600
Epoch 2/10
100/100 [==============================] - 28s 281ms/step - loss: 0.0289 - acc: 0.9910 - val_loss: 0.1288 - val_acc: 0.9660
Epoch 3/10
100/100 [==============================] - 32s 318ms/step - loss: 0.0281 - acc: 0.9895 - val_loss: 0.1283 - val_acc: 0.9650
Epoch 4/10
100/100 [==============================] - 35s 347ms/step - loss: 0.0098 - acc: 0.9965 - val_loss: 0.1337 - val_acc: 0.9610
Epoch 5/10
100/100 [==============================] - 32s 324ms/step - loss: 0.0051 - acc: 0.9980 - val_loss: 0.1362 - val_acc: 0.9640
Epoch 6/10
100/100 [==============================] - 34s 341ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.1476 - val_acc: 0.9630
Epoch 7/10
100/100 [==============================] - 33s 328ms/step - loss: 0.0013 - acc: 0.9995 - val_loss: 0.1768 - val_acc: 0.9650
Epoch 8/10
100/100 [==============================] - 3